In [17]:
import pandas as pd
#import numpy as np
#from math import sqrt
import xgboost

#from sklearn.metrics import mean_squared_error, mean_absolute_error
#from sklearn import metrics
#from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#%matplotlib inline

In [18]:
train_df = pd.read_csv("train.csv", low_memory=False)
target_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [19]:
train_df = train_df[['id','precio']]
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0) #explorar mejores opciones
features.head()

,id,precio,days_to_today,año,antiguedad,centroscomercialescercanos,escuelascercanas,habitaciones,garages,banos,...,Quintana Roo,San luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatán,Zacatecas
0,254099,2273000.0,1540,2015,0.0,0.0,0.0,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,53461,3600000.0,2326,2013,10.0,1.0,1.0,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,247984,1200000.0,1485,2015,5.0,0.0,0.0,3.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,209067,650000.0,2802,2012,1.0,1.0,1.0,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,185997,1150000.0,1251,2016,10.0,0.0,0.0,2.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# target column
y_target = features.precio

# drop 'precio' column
features = features.drop(['id','precio'], axis=1)
features.shape

(240000, 79)

In [22]:
# test size
size_for_test = 0.25

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, y_target, test_size=size_for_test)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(180000, 79) (180000,)
(60000, 79) (60000,)


In [24]:
regressor = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.04,
                             gamma=1, subsample=0.75, colsample_bytree=0.75, max_depth=10)

regressor.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75, gamma=1,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [25]:
preds = regressor.predict(X_test)

In [26]:
df_pred = pd.DataFrame(preds)
df_pred.head()

,0
0,1.265285e+06
1,1.544466e+06
2,4.799604e+05
3,5.948754e+05
4,4.318064e+06


In [27]:
print("Score:", regressor.score(X_test, y_test))

Score: 0.817331875876528


In [34]:
target_df = target_df[['id']]

features_to_pred = pd.merge(target_df, data, on='id', how='inner')
features_to_pred = features_to_pred.fillna(0) #explorar mejores opciones

ids = features_to_pred["id"]

features_to_pred = features_to_pred.drop(['id'], axis=1)

features_to_pred.shape #deberia tener 79 columnas


(60000, 79)

In [35]:
target_prediction = regressor.predict(features_to_pred)
df_predictions = pd.DataFrame(target_prediction)

In [38]:
df_predictions["id"] = ids
df_predictions.rename(columns = {0: 'target'},inplace = True) 
df_predictions = df_predictions[['id','target']]
df_predictions.shape

(60000, 2)

In [39]:
df_predictions.to_csv('ResultadosXGB_rob.csv',index=False)